In [ ]:
import os,cv2,keras
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

In [ ]:
from google.colab.patches import cv2_imshow

In [ ]:
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()

# **calculate intersection over union**

In [ ]:
def get_iou(bb1, bb2):
  assert bb1['x1'] < bb1['x2']
  assert bb1['y1'] < bb1['y2']
  assert bb2['x1'] < bb2['x2']
  assert bb2['y1'] < bb2['y2']

  x1 = max(bb1['x1'], bb2['x1'])
  y1 = max(bb1['y1'], bb2['y1'])

  x2 = min(bb1['x2'], bb2['x2'])
  y2 = min(bb1['y2'], bb2['y2'])

  if x2 < x1 or y2 < y1:
    return 0.0
  intersection_area = (x2 - x1) * (y2 - y1)
  bb1_area = (bb1['x2'] - bb1['x1']) * (bb1['y2'] - bb1['y1'])
  bb2_area = (bb2['x2'] - bb2['x1']) * (bb2['y2'] - bb2['y1'])
  iou = intersection_area / float(bb1_area + bb2_area - intersection_area) # intersection over union
  assert iou >= 0.0
  assert iou <= 1.0
  return iou

# **Load data**

In [ ]:
train_data = []
train_labels = []
couter = 0
for i,image in enumerate(sorted(os.listdir('data/Images'))):
  if "airplane" in image:
    img = cv2.imread('data/Images/'+image)
    train_data.append(img)
for label in sorted(os.listdir('data/Airplanes_Annotations')):
  if "airplane" in label:
    df = pd.read_csv('data/Airplanes_Annotations/'+label)
    t = []
    for index,row in df.iterrows():
      cords_str = row[0].split(" ")
      cord = {"x1":int(cords_str[0]),"y1":int(cords_str[1]),"x2":int(cords_str[2]),"y2":int(cords_str[3])}
      t.append(cord)
    train_labels.append(t)

# **Selective Search**

In [ ]:
def get_countours(train_data, train_labels):
  training_images = []
  training_labels = []

  for i, img in enumerate(train_data):
    try:
      ss.setBaseImage(img)
      ss.switchToSelectiveSearchFast()
      ssresults = ss.process()
      copy_img = img.copy()

      num_positive_samples = 0
      num_negative_samples = 0
      for e, result in enumerate(ssresults):
        if e < 4000:
          x, y, w, h = result
          for gtval in train_labels[i]:
            iou = get_iou(gtval, {"x1": x, "y1": y, "x2": x + w, "y2": y + h})
            if num_positive_samples < 30 and iou > 0.7:
              cuted_image = copy_img[y:y + h, x:x + w]
              resized = cv2.resize(cuted_image, (224, 224), interpolation=cv2.INTER_AREA)
              training_images.append(resized)
              training_labels.append(1)
              num_positive_samples += 1

            if num_negative_samples < 30 and iou < 0.3:
              cuted_image = copy_img[y:y + h, x:x + w]
              resized = cv2.resize(cuted_image, (224, 224), interpolation=cv2.INTER_AREA)
              training_images.append(resized)
              training_labels.append(0)
              num_negative_samples += 1
            if num_negative_samples==30 and num_positive_samples==30:
              break
        else:
          break
    except:
      continue
  return training_images, training_labels



In [ ]:
training_images,training_labels = get_countours(train_data,train_labels)

In [ ]:
X = np.array(training_images)
y = np.array(training_labels)

In [ ]:
from keras.layers import Dense
from keras import Model
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.applications.vgg16 import VGG16
vggmodel = VGG16(weights='imagenet', include_top=True)

553467096/553467096 [==============================] - 24s 0us/step


In [ ]:
for layers in vggmodel.layers[:15]:
  layers.trainable = False
inp = vggmodel.layers[-2].output
predictions = Dense(2, activation='softmax')(inp)
model_final = Model(inputs = vggmodel.input, outputs = predictions)
opt = Adam(lr=0.0001)
model_final.compile(loss = keras.losses.categorical_crossentropy, optimizer = opt, metrics=["accuracy"])
model_final.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
new_Y = []
for val in y :
  t = [0.,0.]
  t[val] = 1.
  new_Y.append(t)
new_Y = np.array(new_Y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, new_Y, test_size=0.1, random_state=42)

In [ ]:
trdata = ImageDataGenerator(horizontal_flip=True, vertical_flip=True, rotation_range=90,)
traindata = trdata.flow(x=X_train, y=y_train,shuffle=True)
tsdata = ImageDataGenerator(horizontal_flip=True, vertical_flip=True, rotation_range=90)
testdata = tsdata.flow(x=X_test, y=y_test,shuffle=True)

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("ieeercnn_vgg16_1.h5", monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_loss', min_delta=0, patience=50, verbose=1, mode='auto')
hist = model_final.fit_generator(generator= traindata, steps_per_epoch= 10, epochs= 500, validation_data= testdata, validation_steps=2, callbacks=[checkpoint,early])

<ipython-input-18-0de1866649d3>:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist = model_final.fit_generator(generator= traindata, steps_per_epoch= 10, epochs= 100, validation_data= testdata, validation_steps=2, callbacks=[checkpoint,early])


Epoch 1/100
10/10 [==============================] - ETA: 0s - loss: 162.0459 - accuracy: 0.4688
Epoch 1: val_loss improved from inf to 1.07591, saving model to ieeercnn_vgg16_1.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


10/10 [==============================] - 30s 2s/step - loss: 162.0459 - accuracy: 0.4688 - val_loss: 1.0759 - val_accuracy: 0.5156
Epoch 2/100
10/10 [==============================] - ETA: 0s - loss: 3.5568 - accuracy: 0.5375
Epoch 2: val_loss improved from 1.07591 to 0.69521, saving model to ieeercnn_vgg16_1.h5
10/10 [==============================] - 21s 2s/step - loss: 3.5568 - accuracy: 0.5375 - val_loss: 0.6952 - val_accuracy: 0.5000
Epoch 3/100
10/10 [==============================] - ETA: 0s - loss: 1.1879 - accuracy: 0.5156
Epoch 3: val_loss did not improve from 0.69521
10/10 [==============================] - 5s 443ms/step - loss: 1.1879 - accuracy: 0.5156 - val_loss: 0.9214 - val_accuracy: 0.6094
Epoch 4/100
10/10 [==============================] - ETA: 0s - loss: 3.3596 - accuracy: 0.5000
Epoch 4: val_loss did not improve from 0.69521
10/10 [==============================] - 10s 1s/step - loss: 3.3596 - accuracy: 0.5000 - val_loss: 0.7479 - val_accuracy: 0.4375
Epoch 5/100
1

In [ ]:
from tensorflow.keras.models import load_model
best_model = load_model("ieeercnn_vgg16_1.h5")

In [ ]:
img = X_train[3]
ss.setBaseImage(img)
ss.switchToSelectiveSearchFast()
ssresults = ss.process()
imout = img.copy()
for e,result in enumerate(ssresults):
    if e < 2000:
        x,y,w,h = result
        timage = imout[y:y+h,x:x+w]
        resized = cv2.resize(timage, (224,224), interpolation = cv2.INTER_AREA)
        img = np.expand_dims(resized, axis=0)
        out= model_final.predict(img,verbose=0)
        print(out)
        if out[0][1] > 0.8:
            cv2.rectangle(imout, (x, y), (x+w, y+h), (0, 255, 0), 1, cv2.LINE_AA)
            plt.figure()
            plt.imshow(imout)
            break